In [1]:
import os
import pandas as pd
from modules.data_preprocessing import DataPreprocessing
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from modules.feature_engineering import FeatureEngineeringV1

In [2]:
data_preprocessing = DataPreprocessing()
fe = FeatureEngineeringV1()

In [3]:
directory_prefix = '../data/excel_raw/'
lst_of_files = [os.path.join(directory_prefix, file) for file in os.listdir(directory_prefix)]
lst_of_files

['../data/excel_raw/20230713T153526004.att.672322581406152.xlsx',
 '../data/excel_raw/20230713T153529650.att.1333037354235214.xlsx']

In [4]:
csv_path='../data/csv/raw.csv'
data_preprocessing.handle_init_files(lst_of_files, csv_path)

The file '../data/csv/raw.csv' already exists. Skipping file creation.


In [5]:
raw = pd.read_csv('../data/csv/raw.csv')
raw

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Route,Ac_id,Ac_code,Ac_type,Ac_kts,Time_step,Phase,Ac_Lat,Ac_Lon,Ac_feet
0,3.2,130.9,1.1,130.3,2.4,202.5,0.8,142.5,1.6,141.7,...,6,R6-VJ150-21,VJ150,1,170.0,3,1,10.8134,106.6427,625.0
1,3.2,130.9,1.1,130.4,2.4,202.9,0.8,142.6,1.6,141.5,...,6,R6-VJ150-21,VJ150,1,179.0,23,1,10.8075,106.6271,1425.0
2,3.2,130.9,1.1,130.4,2.4,202.9,0.8,142.6,1.6,141.5,...,6,R6-VJ150-21,VJ150,1,205.0,43,1,10.8029,106.6096,1800.0
3,3.2,130.9,1.1,130.5,2.4,203.3,0.7,142.8,1.6,141.4,...,6,R6-VJ150-21,VJ150,1,231.0,59,1,10.8063,106.5941,2025.0
4,3.2,130.9,1.1,130.5,2.4,203.3,0.7,142.8,1.6,141.4,...,6,R6-VJ150-21,VJ150,1,255.0,75,1,10.8204,106.5810,2475.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61383,1.9,137.2,1.8,42.6,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,157.0,5344,1,21.2532,105.6876,1775.0
61384,1.9,137.2,1.8,42.6,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,152.0,5362,1,21.2493,105.7010,1525.0
61385,1.9,137.2,1.8,42.2,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,146.0,5378,1,21.2461,105.7121,1325.0
61386,1.9,137.2,1.8,42.2,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,131.0,5412,1,21.2401,105.7331,950.0


In [6]:
# processed_df = fe.process_data('../data/csv/raw.csv')
# processed_df

In [7]:
scalers = {
			'wind_speed': MinMaxScaler(),
			'wind_dir'  : MinMaxScaler(),
			'lat_lon'   : MinMaxScaler(feature_range=(-1,1)),
			'altitude'  : MinMaxScaler(),
			'speed'     : MinMaxScaler(),
			'time_step' : MinMaxScaler(),
		}
encoders = {
			'ac_type': OneHotEncoder(),
			'phase'  : OneHotEncoder(),
		}

In [8]:
def common_preprocessing(df):
	df = df.dropna()
	df = df.drop_duplicates().reset_index(drop=True)
	return df


def normalize_features(df):
	wind_speed_columns = df.columns[df.columns.str.startswith('wind_speed')]
	wind_dir_columns = df.columns[df.columns.str.startswith('wind_dir')]
	lat_lon_columns = ['Ac_Lat', 'Ac_Lon']
	altitude_columns = ['Ac_feet']
	speed_columns = ['Ac_kts']
	time_step_column = ['Time_step']
	
	df[wind_speed_columns] = scalers['wind_speed'].fit_transform(df[wind_speed_columns])
	df[wind_dir_columns] = scalers['wind_dir'].fit_transform(df[wind_dir_columns])
	df[lat_lon_columns] = scalers['lat_lon'].fit_transform(df[lat_lon_columns])
	df[altitude_columns] = scalers['altitude'].fit_transform(df[altitude_columns])
	df[speed_columns] = scalers['speed'].fit_transform(df[speed_columns])
	df[time_step_column] = scalers['time_step'].fit_transform(df[time_step_column])
	
	return df


def label_features(df):
	ac_type = ['Ac_type']
	phase = ['Phase']
	df[ac_type] = encoders['ac_type'].fit_transform(df[['Ac_type']])
	df[phase] = encoders['phase'].fit_transform(df[['Phase']])
	
	return df


def remove_unreasonable_time(df, max_threshold=10000, min_flight_time=4000):
	cleaned_df = df[df['Time_step'] < max_threshold]
	flights_to_remove = cleaned_df.groupby('Ac_id')['Time_step'].max() < min_flight_time
	id_to_remove = flights_to_remove[flights_to_remove].index
	cleaned_df = cleaned_df[~cleaned_df['Ac_id'].isin(id_to_remove)]
	return cleaned_df

In [9]:
processed_df = common_preprocessing(raw)
processed_df

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Route,Ac_id,Ac_code,Ac_type,Ac_kts,Time_step,Phase,Ac_Lat,Ac_Lon,Ac_feet
0,3.2,130.9,1.1,130.3,2.4,202.5,0.8,142.5,1.6,141.7,...,6,R6-VJ150-21,VJ150,1,170.0,3,1,10.8134,106.6427,625.0
1,3.2,130.9,1.1,130.4,2.4,202.9,0.8,142.6,1.6,141.5,...,6,R6-VJ150-21,VJ150,1,179.0,23,1,10.8075,106.6271,1425.0
2,3.2,130.9,1.1,130.4,2.4,202.9,0.8,142.6,1.6,141.5,...,6,R6-VJ150-21,VJ150,1,205.0,43,1,10.8029,106.6096,1800.0
3,3.2,130.9,1.1,130.5,2.4,203.3,0.7,142.8,1.6,141.4,...,6,R6-VJ150-21,VJ150,1,231.0,59,1,10.8063,106.5941,2025.0
4,3.2,130.9,1.1,130.5,2.4,203.3,0.7,142.8,1.6,141.4,...,6,R6-VJ150-21,VJ150,1,255.0,75,1,10.8204,106.5810,2475.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61380,1.9,137.2,1.8,42.6,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,157.0,5344,1,21.2532,105.6876,1775.0
61381,1.9,137.2,1.8,42.6,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,152.0,5362,1,21.2493,105.7010,1525.0
61382,1.9,137.2,1.8,42.2,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,146.0,5378,1,21.2461,105.7121,1325.0
61383,1.9,137.2,1.8,42.2,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,131.0,5412,1,21.2401,105.7331,950.0


In [10]:
processed_df = remove_unreasonable_time(processed_df)
processed_df

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Route,Ac_id,Ac_code,Ac_type,Ac_kts,Time_step,Phase,Ac_Lat,Ac_Lon,Ac_feet
0,3.2,130.9,1.1,130.3,2.4,202.5,0.8,142.5,1.6,141.7,...,6,R6-VJ150-21,VJ150,1,170.0,3,1,10.8134,106.6427,625.0
1,3.2,130.9,1.1,130.4,2.4,202.9,0.8,142.6,1.6,141.5,...,6,R6-VJ150-21,VJ150,1,179.0,23,1,10.8075,106.6271,1425.0
2,3.2,130.9,1.1,130.4,2.4,202.9,0.8,142.6,1.6,141.5,...,6,R6-VJ150-21,VJ150,1,205.0,43,1,10.8029,106.6096,1800.0
3,3.2,130.9,1.1,130.5,2.4,203.3,0.7,142.8,1.6,141.4,...,6,R6-VJ150-21,VJ150,1,231.0,59,1,10.8063,106.5941,2025.0
4,3.2,130.9,1.1,130.5,2.4,203.3,0.7,142.8,1.6,141.4,...,6,R6-VJ150-21,VJ150,1,255.0,75,1,10.8204,106.5810,2475.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61380,1.9,137.2,1.8,42.6,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,157.0,5344,1,21.2532,105.6876,1775.0
61381,1.9,137.2,1.8,42.6,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,152.0,5362,1,21.2493,105.7010,1525.0
61382,1.9,137.2,1.8,42.2,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,146.0,5378,1,21.2461,105.7121,1325.0
61383,1.9,137.2,1.8,42.2,5.3,37.8,3.4,95.0,3.9,97.4,...,6,R6-VN258-04,VN258,2,131.0,5412,1,21.2401,105.7331,950.0


In [11]:
processed_df = normalize_features(processed_df)
processed_df

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Route,Ac_id,Ac_code,Ac_type,Ac_kts,Time_step,Phase,Ac_Lat,Ac_Lon,Ac_feet
0,0.928702,0.495425,-0.634564,0.022136,-1.145864,2.972182,-2.250708,5.614973,-1.593272,2.974218,...,6,R6-VJ150-21,VJ150,1,-2.268434,-1.526071,1,-1.407072,-0.227452,-1.853900
1,0.928702,0.495425,-0.634564,0.023238,-1.145864,2.980370,-2.250708,5.623375,-1.593272,2.963249,...,6,R6-VJ150-21,VJ150,1,-2.183981,-1.515271,1,-1.408670,-0.245917,-1.791555
2,0.928702,0.495425,-0.634564,0.023238,-1.145864,2.980370,-2.250708,5.623375,-1.593272,2.963249,...,6,R6-VJ150-21,VJ150,1,-1.940006,-1.504471,1,-1.409917,-0.266631,-1.762331
3,0.928702,0.495425,-0.634564,0.024339,-1.145864,2.988558,-2.315868,5.640178,-1.593272,2.957764,...,6,R6-VJ150-21,VJ150,1,-1.696031,-1.495831,1,-1.408996,-0.284978,-1.744796
4,0.928702,0.495425,-0.634564,0.024339,-1.145864,2.988558,-2.315868,5.640178,-1.593272,2.957764,...,6,R6-VJ150-21,VJ150,1,-1.470824,-1.487191,1,-1.405175,-0.300484,-1.709727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61380,-0.413190,0.599672,0.090406,-0.943672,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,6,R6-VN258-04,VN258,2,-2.390422,1.358068,1,1.422109,-1.357957,-1.764279
61381,-0.413190,0.599672,0.090406,-0.943672,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,6,R6-VN258-04,VN258,2,-2.437340,1.367788,1,1.421052,-1.342096,-1.783762
61382,-0.413190,0.599672,0.090406,-0.948077,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,6,R6-VN258-04,VN258,2,-2.493642,1.376428,1,1.420185,-1.328957,-1.799348
61383,-0.413190,0.599672,0.090406,-0.948077,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,6,R6-VN258-04,VN258,2,-2.634397,1.394788,1,1.418559,-1.304101,-1.828572


In [12]:
processed_df = label_features(processed_df)
processed_df

,wind_speed_TSN,wind_dir_TSN,wind_speed_KADUM,wind_dir_KADUM,wind_speed_PATNA,wind_dir_PATNA,wind_speed_SADAS,wind_dir_SADAS,wind_speed_DADEN,wind_dir_DADEN,...,Route,Ac_id,Ac_code,Ac_type,Ac_kts,Time_step,Phase,Ac_Lat,Ac_Lon,Ac_feet
0,0.928702,0.495425,-0.634564,0.022136,-1.145864,2.972182,-2.250708,5.614973,-1.593272,2.974218,...,6,R6-VJ150-21,VJ150,"(0, 0)\t1.0",-2.268434,-1.526071,"(0, 0)\t1.0",-1.407072,-0.227452,-1.853900
1,0.928702,0.495425,-0.634564,0.023238,-1.145864,2.980370,-2.250708,5.623375,-1.593272,2.963249,...,6,R6-VJ150-21,VJ150,"(0, 0)\t1.0",-2.183981,-1.515271,"(0, 0)\t1.0",-1.408670,-0.245917,-1.791555
2,0.928702,0.495425,-0.634564,0.023238,-1.145864,2.980370,-2.250708,5.623375,-1.593272,2.963249,...,6,R6-VJ150-21,VJ150,"(0, 0)\t1.0",-1.940006,-1.504471,"(0, 0)\t1.0",-1.409917,-0.266631,-1.762331
3,0.928702,0.495425,-0.634564,0.024339,-1.145864,2.988558,-2.315868,5.640178,-1.593272,2.957764,...,6,R6-VJ150-21,VJ150,"(0, 0)\t1.0",-1.696031,-1.495831,"(0, 0)\t1.0",-1.408996,-0.284978,-1.744796
4,0.928702,0.495425,-0.634564,0.024339,-1.145864,2.988558,-2.315868,5.640178,-1.593272,2.957764,...,6,R6-VJ150-21,VJ150,"(0, 0)\t1.0",-1.470824,-1.487191,"(0, 0)\t1.0",-1.405175,-0.300484,-1.709727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61380,-0.413190,0.599672,0.090406,-0.943672,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,6,R6-VN258-04,VN258,"(0, 1)\t1.0",-2.390422,1.358068,"(0, 0)\t1.0",1.422109,-1.357957,-1.764279
61381,-0.413190,0.599672,0.090406,-0.943672,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,6,R6-VN258-04,VN258,"(0, 1)\t1.0",-2.437340,1.367788,"(0, 0)\t1.0",1.421052,-1.342096,-1.783762
61382,-0.413190,0.599672,0.090406,-0.948077,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,6,R6-VN258-04,VN258,"(0, 1)\t1.0",-2.493642,1.376428,"(0, 0)\t1.0",1.420185,-1.328957,-1.799348
61383,-0.413190,0.599672,0.090406,-0.948077,-0.007508,-0.399230,-0.556559,1.624079,0.149889,0.544497,...,6,R6-VN258-04,VN258,"(0, 1)\t1.0",-2.634397,1.394788,"(0, 0)\t1.0",1.418559,-1.304101,-1.828572


In [ ]:
decoded_df = fe.decode_features(processed_df.copy())
decoded_df

In [ ]:
processed_df.to_csv('../data/csv/processed.csv', index=False)